# The prior of the linear model
Now that we have the likelihood, let's have a look at the prior. We have three parameters in our model and likelihood function: $\beta_0$, $\beta_1$ and $\sigma$. Again, the prior reflect the belief we have about which values are most likely for each parameters. So you might already think about what that could look like. If you have some knowledge about what values $\beta_0$, $\beta_1$ could look like, you can imagine that the prior should be a distribution centered around those values for those parameters, with max likelihood around those values and decreasing when you go lower or higher than that. 

So yeah, some sort of a normal distribution of sorts. Typically, you wouldn't have a guess about the precise value of the parameters, but you'd know that there are some values that are more realistic than others. So you can specify the priors for these parameters with some mean and variance parameters that kind of make sense. For our particular example, since you are wonder whether there is a relationship between penguin flipper length and their weight, you can imagine that the values around 0 should be quite likely and that very large values are less likely. And for the $\beta_0$ parameter, even if you don't know the actual mean weight of penguin, you know it's probably not 500g and probably not 750Kg , so something in between. So you can play around with the normal distribution to find parameters that seem reasonable (that's how I got the parameters below):

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Functions created in previous sections:
def normal_pdf(x, mu, sigma):
    p_x = (1/(np.sqrt(2*np.pi*sigma**2))) * np.e**(-((x-mu)**2)/(2*sigma**2))
    return p_x



# Parameters for the beta0 and beta1 prior distributions:
b0_mu = 30  # I think a penguin weighting 30kg seems about right
b0_sig = 15  # To set this parameter, I played around until I got a distribution that seem reasonable
b1_mu = 0  # I think the values of b1 should be close to 0. I think there may be a relationship between the two variables, but I dont' know if it's a positive or a negative relationship, which means that positive and negative values are equally likely a priori
b1_sig = 5  # This is quite large, which means that many values of b1 are plausible

xs = np.arange(-5, 5, 0.1)  # Define values of x

fig, ax = plt.subplots(2)
xs = np.arange(b0_mu-50, b0_mu+50, 0.1)  # Define values of x
ax[0].plot(xs, [normal_pdf(x, b0_mu, b0_sig) for x in xs], label=f'$\\mu={b0_mu}$, $\\sigma={b0_sig}$')
ax[0].set_ylabel('P($\\beta_0$)')
ax[0].set_xlabel('$\\beta_0$')
ax[0].legend()

xs = np.arange(b1_mu-10, b1_mu+10, 0.1)  # Define values of x
ax[1].plot(xs, [normal_pdf(x, b1_mu, b1_sig) for x in xs], label=f'$\\mu={b1_mu}$, $\\sigma={b1_sig}$')
ax[1].set_ylabel('P($\\beta_1$)')
ax[1].set_xlabel('$\\beta_1$')
ax[1].legend()
plt.tight_layout()
plt.show()
plt.close()

NameError: name 'np' is not defined

These seems about right: I think most penguin weight less than 80Kg, and I don't think that the correlation between the flipper length and the penguin weight will be much more than 10 or -10, cause that would mean that for every increase of 1mm in penguin flipper length, they gain 10kg, that just feels a bit much. Are these priors good? This is one of the main reason people are skeptical about Bayesian statistics: the definition of the priors influence the posterior. Since the priors can be set up arbitrarily, some people argue that this opens up the door for subjectivity in the analysis and that we can tweak the priors to get any results we want. We will see later on that there are ways in which we can make sure that that's not the case and that our analysis is sound. But in this cpahter I will stick to picking values that seem reasonable. 

What about the $\sigma$ parameter? Well in that case, picturing something that's normally distributed would be wrong. That's because the $\sigma$ parameter has to remain positive. That's pretty logical. If the true value of $\sigma$ were zero, then you'd have zero measurement noise and the observed data would fall perfectly on your regression line. If the true value of $\sigma$ is larger than 0, then you have some error. That doesn't leave any room for a negative $\sigma$, does it? In other words, we know a priori that $\sigma$ should be 0 or larger. So we need a function that has a probability of 0 below 0. And just as was the case for the $\beta$s parameters, eventhough we don't have a precise guess for what $\sigma$ is, we can guess a range: it's not very likely that we have a $\sigma$ of 10000, because that would mean there is so much noise that we wouldn't find anything anyways. But once again, it feels like there are many kind of distributions that could be up for the task, but which one should we choose? The answer is once again: whatever is easier to work with and combine with the other distributions from a mathematical standpoint. One such distribution that is often used is the inverse gamma distribution:

$$\sigma^2 \sim \mathcal{Inverse-Gamma}(a, b)$$

And the formulae of the inverse gamma is the following:

$$f(x: \alpha, \beta) = \frac{\beta^\alpha}{\Gamma(\alpha)}(1/x)^{\alpha+1}exp(-\beta/x)$$

So let's write that in python

In [ ]:
from math import gamma


def inv_gamma_pdf(x, alpha, beta):
    """
    Computes the probability density function of the inverse gamma distribution.
    
    Parameters:
    - x : float or np.ndarray
        The value(s) at which to evaluate the PDF. Must be positive.
    - alpha : float
        The shape parameter of the inverse gamma distribution. Must be positive.
    - beta : float
        The scale parameter of the inverse gamma distribution. Must be positive.
        
    Returns:
    - float or np.ndarray
        The PDF of the inverse gamma distribution evaluated at x.
    
    Notes:
    The inverse gamma distribution PDF is given by:
    
        f(x; alpha, beta) = (beta ** alpha / gamma(alpha)) * x ** (-alpha - 1) * exp(-beta / x)
    
    where `alpha` > 0 and `beta` > 0.
    """
    return (x ** (-alpha-1) / gamma(alpha)) * np.exp(-1 / x)

It's related to the $\beta$ distribution we saw in the previous chapter (not to be confused with the distribution of our $\beta$ parameter now), and it takes two parameters: $\alpha$ and $\beta$, which control the shape of the distribution. It's not as direct as the mean and the spread parameter of the normal distribution which are expressed directly in the units we are interested in, but we can just play around until we find values that seem reasonable:

In [ ]:
from scipy.stats import invgamma

xs = np.linspace(0, 3, 1000)  # Define values of x
mu = 0
sigma = 1
fig, ax = plt.subplots()
ax.plot(xs, [inv_gamma_pdf(x, 1, 1) for x in xs], label=f'$\\alpha={1}$, $\\beta={1}$')
ax.set_ylabel('P($\\sigma$)')
ax.set_xlabel('$\\sigma$')
ax.legend()
plt.tight_layout()
plt.show()
plt.close()

I ended up with 1 and 1, seems about reasonable: small variance values are most likely, but larger ones are also not too unlikely. It feels right to me and that is what matters. Typically, you would not just randomly pick values that seem to make sense for this one, but instead base these parameters following some standards and conventions (which I will discuss later in the book), but for now, given we are just trying to illustrate how things work, that's good enough.

## Combining the priors of each parameter together

Above, we presented what reasonable priors for each parameters would be in isolation. But what we need ultimately is the prior distribution of all our parameters. Should we combine them somehow, multiply them...? We saw before that we can't simply multiply the likelihood of $\beta_0$ and $\beta_1$ by multiplying them, does that also apply here? The answer is: it depends. For the likelihood, we couldn't simply multiply the likelihood of $P(y|\beta_0)$ and $P(y|\beta_1)$, because that would imply that the likelihood of the data under each parameter is independent. And we saw that that's not necessarily the case, and that depending on the correlation between $\beta_0$ and $beta_1$, the likelihood can be really different. And so if we want to know the likelihood of the data given the parameters, we need to model the likelihood as a function of all three parameters combined, otherwise we may get a misrepresentation of the true likelihood (depending on whether there is a correlation between the paramters for the liklelihood). 

For the priors however, we are expressing our belief in the likely values of $\beta_0$ and $\beta_1$ and $\sigma$. So whether the final prior distribution is obtained by multiplying the probability distribution of each parameter independently also depends on hour belief. Remember, when you multiply two probabilities, you assume that the two variables following these probability distributions are independent. So if you believe that your parameters are independent from each other, you should create a prior for each independently and multiply them. But if you don't believe that they are independent, you should instead specify your prior as a multivariate distribution, specifying the probability of one variable by taking into account its relationship with the probability distribution of the other variables. 

This may sound a little bit confusing. So let's make it a bit clearer with one example. Say we had a model with two different regressors: flipper length, but also feet length of our penguin. You would assume a priori that on top of the potential relationship between penguin weight and flipper length and the relationship between penguin weight and their feet size, there is probably also a positive correlation between penguin flipper length and their feety size: penguins with longer flippers tend to have longer feet. If that's the case, you should specify your prior such that they reflect that belief. 

This is just an example. The point is that you have the choice: specifying the priors as multivariate distributions or as univariate distributions that get multiplied with each other. Importantly, you will in most cases do a mixture of both options, because no matter what, you will never have a prior that assume that there is a correlation between the error and the $\beta$ distribution, because that would be violating the main assumption of the linear models, that the error is i.i.d. Furthermore, for the purpose of this book, we will take a commitment: you will always specify the priors of the $\betas$ as a multivariate distribution. That's because its more general: if you specify the priors for the beta as a multivariate distribution, you can still set the betas to be independent from each other: simply set the covariance between these parameters to 0.

Okay, so with all that being said, let's write down the formulae for our priors: